In [1]:

import cobra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
model = cobra.io.read_sbml_model("../model/Mouse-GEM.xml")

https://identifiers.org/taxonomy/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


In [ ]:
# Generating model summary
import pandas as pd

model_summary = {
    "Organism": "Mus musculus",
    "Model version": "Mouse-GEM v1.8.0",
    "Reactions": len(model.reactions),
    "Metabolites": len(model.metabolites),
    "Genes": len(model.genes),
    "Compartments": ", ".join(model.compartments.values()),
    "Objective reaction": model.objective.expression
}

summary_df = pd.DataFrame.from_dict(model_summary, orient="index", columns=["Value"])
summary_df


,Value
Organism,Mus musculus
Model version,Mouse-GEM v1.8.0
Reactions,12987
Metabolites,8454
Genes,2846
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic ..."
Objective reaction,1.0*MAR00021 - 1.0*MAR00021_reverse_97974


In [4]:
summary_df.to_csv("Appendix_A_Model_Summary.csv")


# B Exchange reactions and environmental bounds

In [5]:
#Provide transparency on nutrients/media definition
exchange_data = pd.DataFrame({
    "Reaction ID": [rxn.id for rxn in model.exchanges],
    "Equation": [rxn.build_reaction_string(use_metabolite_names=True) for rxn in model.exchanges],
    "Lower bound": [rxn.lower_bound for rxn in model.exchanges],
    "Upper bound": [rxn.upper_bound for rxn in model.exchanges],
})

exchange_data.head()


,Reaction ID,Equation,Lower bound,Upper bound
0,MAR07108,benzo[a]pyrene <=>,-1000.0,1000.0
1,MAR07110,naphthalene <=>,-1000.0,1000.0
2,MAR07112,aflatoxin B1 <=>,-1000.0,1000.0
3,MAR07114,trichloroethene <=>,-1000.0,1000.0
4,MAR07116,bromobenzene <=>,-1000.0,1000.0


In [6]:
exchange_data.to_csv("Appendix_B_Exchange_Reactions.csv", index=False)


# C Active reactions under biomass optimisation

In [7]:
#List reactions carrying non-zero flux under biomass FBA.
solution = model.optimize()

fluxes = solution.fluxes
active_reactions = fluxes[fluxes != 0]

active_df = active_reactions.reset_index()
active_df.columns = ["Reaction ID", "Flux"]

len(active_df)  # should be ~1188

active_df["Reaction name"] = [
    model.reactions.get_by_id(rid).name for rid in active_df["Reaction ID"]
]

active_df.head()


,Reaction ID,Flux,Reaction name
0,MAR04281,4.762641,(S)-Lactate:NAD+ oxidoreductase
1,MAR08357,1000.000000,Acetaldehyde:NAD+ oxidoreductase
2,MAR04358,994.884606,ATP:pyruvate 2-O-phosphotransferase
3,MAR04363,1000.000000,2-phospho-D-glycerate hydro-lyase (phosphoenol...
4,MAR04365,-1000.000000,"D-phosphoglycerate 2,3-phosphomutase"


In [8]:
active_df.to_csv("Appendix_C_Active_Reactions.csv", index=False)


# D Singel-gene deletions results


In [9]:
#Full dataset behind gene essentiality analysis.
from cobra.flux_analysis import single_gene_deletion

gene_deletion_results = single_gene_deletion(model)
gene_deletion_results.head()


,ids,growth,status
0,{Txnrd2},4.262828,optimal
1,{Plpp3},4.262828,optimal
2,{Uckl1},4.262828,optimal
3,{Dolk},4.262828,optimal
4,{Chst1},4.262828,optimal


In [10]:
gene_deletion_results.to_csv("Appendix_D_Single_Gene_Deletion_Raw.csv", index=False)


In [11]:
# Classifying gene essentiality table
baseline_growth = round(model.optimize().objective_value, 5)

df = gene_deletion_results.copy()
df["growth"] = df["growth"].round(5)

essential = df[df["growth"] == 0]
partial = df[(df["growth"] > 0) & (df["growth"] < baseline_growth)]
nonessential = df[df["growth"] == baseline_growth]

classification_df = pd.DataFrame({
    "Category": ["Essential", "Partially essential", "Non-essential"],
    "Number of genes": [len(essential), len(partial), len(nonessential)],
    "Percentage (%)": [
        100 * len(essential) / len(df),
        100 * len(partial) / len(df),
        100 * len(nonessential) / len(df)
    ]
})

classification_df


,Category,Number of genes,Percentage (%)
0,Essential,95,3.338018
1,Partially essential,37,1.300070
2,Non-essential,2714,95.361911


In [12]:
classification_df.to_csv("Appendix_D_Gene_Essentiality_Summary.csv", index=False)


# E Reproducibility scripts


In [13]:
#Biomass FBA
model.objective = "MAR00021"
solution = model.optimize()
solution.objective_value


4.262827945443597

In [15]:
#Anaerobic simulation
anaerobic_model = model.copy()
anaerobic_model.reactions.get_by_id("MAR09048").lower_bound = 0  # block O2
anaerobic_model.optimize().objective_value


4.262827945443598

In [16]:
#Oxygen + CO₂ blocked
strict_model = model.copy()
strict_model.reactions.get_by_id("MAR09048").lower_bound = 0  # O2
strict_model.reactions.get_by_id("MAR09058").upper_bound = 0  # CO2
strict_model.optimize().objective_value


4.262827945443599

In [17]:
#Nutrient blocking loop (uptake sensitivity)
uptake_reactions = [rxn for rxn in model.exchanges if rxn.lower_bound < 0]

results = []

for rxn in uptake_reactions:
    original_lb = rxn.lower_bound
    rxn.lower_bound = 0
    sol = model.optimize()
    results.append({
        "Reaction": rxn.id,
        "Biomass": sol.objective_value if sol.status == "optimal" else 0
    })
    rxn.lower_bound = original_lb

uptake_df = pd.DataFrame(results)
uptake_df.to_csv("Appendix_E_Uptake_Blocking.csv", index=False)
